In [3]:
# Overloading a function allows us to create two equally named ones
# as long as the distinction can be made from the number and name of the parameters

# in python we don't have this since we don't have static typing, instead we could 
# use dispatch which follows an approach to get the first argument type and check.

from html import escape
from decimal import Decimal

In [4]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>'

def html_real(a):
    return f'{round(a, 2): .2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l): # sequence type
  items = (f'<li>{html_escape(item)}</li>' for item in l)
  return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(l): # mapping type
  items = (f'<li>{k}={v}</li>' for k, v in l.items())
  return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [5]:
def htmlize(arg):
  if isinstance(arg, int):
    return html_int(arg)
  elif isinstance(arg, float) or isinstance(arg, Decimal):
    return html_real(arg)
  elif isinstance(arg, str):
    return html_str(arg)
  elif isinstance(arg, list) or isinstance(arg, tuple):
    return html_list(arg)
  elif isinstance(arg, dict):
    return html_dict(arg)
  else:
    return html_escape(arg)
  

In [7]:
htmlize(100)

'a'

In [8]:
htmlize("""python is 
awesome""")

'python is <br/>\nawesome'

In [10]:
print(htmlize([1,2,3]))
# however the approach is not as straightforwar, we can see that by using htmlize on 
# a list of multiple types we don't actually perform on each individual type
print(htmlize(["""hello
world!
""", (10, 20, 30), 100]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>
<ul>
<li>hello
world!
</li>
<li>(10, 20, 30)</li>
<li>100</li>
</ul>


In [12]:
# in python we are actually allowed to define a function that references another not
# yet defined function as long as we actually create the function before running.
def fun1():
  fun2()

def fun2():
  print('hello')

In [13]:
# So keeping that in mind we can call htmlize on the actual function for the list
# to htmlize the contents inseide the list
def htmlize(arg):
  if isinstance(arg, int):
    return html_int(arg)
  elif isinstance(arg, float) or isinstance(arg, Decimal):
    return html_real(arg)
  elif isinstance(arg, str):
    return html_str(arg)
  elif isinstance(arg, list) or isinstance(arg, tuple):
    return html_list(arg)
  elif isinstance(arg, dict):
    return html_dict(arg)
  else:
    return html_escape(arg)

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>'

def html_real(a):
    return f'{round(a, 2): .2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l): # sequence type
  items = (f'<li>{htmlize(item)}</li>' for item in l)
  return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(l): # mapping type
  items = (f'<li>{html_escape(k)}={htmlize(v)}</li>' for k, v in l.items())
  return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [14]:
# now it works.
print(htmlize(["""hello
world!
""", (10, 20, 30), 100]))

<ul>
<li>hello<br/>
world!<br/>
</li>
<li><ul>
<li>10(<i>0xa</i></li>
<li>20(<i>0x14</i></li>
<li>30(<i>0x1e</i></li>
</ul></li>
<li>100(<i>0x64</i></li>
</ul>


In [19]:
# however the way we are creating our code means that each time we want to add a
# custom class to htmlize we have to add the function to handle and then add
# the entry to htmlize which would get pretty out of hand quick.


# this approach adds a registry, though it is cleaner we still have to go back
# to this function and add a new object when created.
def htmlize(arg):
  registry = {
      object: html_escape,
      int: html_int,
      float: html_real,
      Decimal: html_real,
      list: html_list,
      tuple: html_list,
      dict: html_dict 
  }
  fn = registry.get(type(arg), registry[object])
  return fn(arg)

# now it works.
print(htmlize(["""hello
world!
""", (10, 20, 30), 100]))

<ul>
<li>hello
world!
</li>
<li><ul>
<li>10(<i>0xa</i></li>
<li>20(<i>0x14</i></li>
<li>30(<i>0x1e</i></li>
</ul></li>
<li>100(<i>0x64</i></li>
</ul>


In [19]:
# let's then use a decorator to write the same function so it will allow us to
# implement the fixes.
def singledispatch(fn):
  registry = {}
  registry[object] = fn
  def decorated(arg):
    return registry.get(type(arg), registry[object])(arg)

  def register(type_):
    def inner(fn):
      registry[type_] = fn
      return fn
    return inner

  def dispatch(type_):
    return registry.get(type_, registry[object])

  decorated.register = register
  decorated.registry = registry # useful for debug but not to leave to end user
  decorated.dispatch = dispatch

  return decorated

@singledispatch
def htmlize(a):
  return escape(str(a))

print(htmlize('1 < 100'))
# we can now register functions to the decorator.


@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>'

@htmlize.register(float)
def html_real(a):
    return f'{round(a, 2): .2f}'

@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_list(l): # sequence type
  items = (f'<li>{htmlize(item)}</li>' for item in l)
  return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(l): # mapping type
  items = (f'<li>{html_escape(k)}={htmlize(v)}</li>' for k, v in l.items())
  return '<ul>\n' + '\n'.join(items) + '\n</ul>'


1 &lt; 100


In [15]:
# what we are actually doing
html_list = htmlize.register(list)(html_list)

# NOTE. isinstance recognizes subclasses.

'<ul>\n<li>a</li>\n<li>s</li>\n<li>s</li>\n<li>s</li>\n<li>s</li>\n</ul>'

In [20]:
print(htmlize.dispatch(int))

<function html_int at 0x7f6deb4afb90>


In [21]:
# now the python version
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence

In [26]:
# we can think of this first function as being the default for the decorator.
@singledispatch
def htmlize(a):
  return escape(str(a))

@htmlize.register(Integral)
def html_int(a):
  return f'{a}(<i>{str(hex(a))}</i>'
# Check the registry
htmlize.registry
# Check the function for the type
htmlize.dispatch(int) # now it works for all instances of Integral.
htmlize.dispatch(bool) # also isinstance of Integral.


<function __main__.html_int>

In [29]:
# We do have a problem with strings since they are str but also Sequences
isinstance('str', Sequence)

# to fix this with the built in we just register for str, in the registry
# it will check the class inheritance hierarchy and find the closest one.
# so str is close to str thatn to Sequence.
@htmlize.register(str)
def _(s): # we don't ultimately care about the label here since we call by
    # the instance object being passed.
    return html_escape(s).replace('\n', '<br/>\n')